In [1]:
import pandas as pd
import datetime
from loguru import logger
from sql import PGSQL

pgsql = PGSQL()

2025-09-11 11:24:18.236 | INFO     | sql:sql_to_df:53 - | tdm_data | Query on database "portal": "SELECT * FROM reporting.get_failure_summary(:prod_date, :shift)"
2025-09-11 11:24:18.237 | INFO     | sql:sql_to_df:54 - | tdm_data | Params: "{'prod_date': '2025-09-08', 'shift': 1}"
2025-09-11 11:24:23.485 | INFO     | sql:sql_to_df:67 - | tdm_data | Executed: 0:00:05.248142


In [4]:
sql = """SELECT * FROM reporting.get_failure_summary(:prod_date, :shift)"""
params = {
    'prod_date': '2025-09-08',
    'shift': 1
}

In [5]:
%%time
df = pgsql.sql_to_df(query=sql, params=params, db='portal', mod='tdm_data')

2025-09-11 11:24:20,333 DEBUG sqlalchemy.pool.impl.QueuePool Created new connection <connection object at 0x0000029550E8FEF0; dsn: 'user=postgres password=xxx dbname=portal host=10.121.220.126 port=5432', closed: 0>
2025-09-11 11:24:22,673 DEBUG sqlalchemy.pool.impl.QueuePool Connection <connection object at 0x0000029550E8FEF0; dsn: 'user=postgres password=xxx dbname=portal host=10.121.220.126 port=5432', closed: 0> checked out from pool
2025-09-11 11:24:23,483 DEBUG sqlalchemy.pool.impl.QueuePool Connection <connection object at 0x0000029550E8FEF0; dsn: 'user=postgres password=xxx dbname=portal host=10.121.220.126 port=5432', closed: 0> being returned to pool
2025-09-11 11:24:23,484 DEBUG sqlalchemy.pool.impl.QueuePool Connection <connection object at 0x0000029550E8FEF0; dsn: 'user=postgres password=xxx dbname=portal host=10.121.220.126 port=5432', closed: 0> rollback-on-return
CPU times: total: 156 ms
Wall time: 5.25 s


In [7]:
summary_df = df.copy()

# Clean and sort data
summary_df['failed_pct'] = summary_df['failed'] / summary_df['total']
summary_df = summary_df.sort_values(by='failed_pct', ascending=False)
summary_df = summary_df.drop(columns=['failed_percentage'])
col_order = ['tool', 'total', 'failed', 'failed_pct', 'max_nok',
             'no_effort', 'tool_use', 'bypass_reset']
summary_df = summary_df[col_order]

In [8]:
summary_df

,tool,total,failed,failed_pct,max_nok,no_effort,tool_use,bypass_reset
64,F2-07R RH FRONT STRUT BAR AT FEM,16,16,1.000000,0,0,16,0
14,C2-11L ICE TM Mnt Brkt to Body & TM,8,8,1.000000,0,0,0,0
15,C2-14L EV RrGRD_ICE TrnsGRD_L+RH Mfl Hng,29,11,0.379310,0,0,0,0
120,T2-05RB - ROOF RIGID BOLTS (RH),8,3,0.375000,0,0,0,0
70,F2-11LB LH 3rd Row Seat Belt LWR Anchor,221,71,0.321267,0,0,0,0
...,...,...,...,...,...,...,...,...
105,PDI - Alignment Pit 2 Front,184,0,0.000000,0,0,0,0
63,F2-07LC JK EV & JK ICE AC LINES TO FEM,247,0,0.000000,0,0,0,0
65,F2-08L LH FRONT STRUT BAR AT FEM,26,0,0.000000,0,0,0,0
122,T2-06LB - ROOF RIGID BOLTS (LH),8,0,0.000000,0,0,0,0
